Cleaner version of the notebook that I run on laptop to scape data faster

In [14]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os

In [22]:
#captcha alert settings
import winsound
duration = 5000  # milliseconds
freq = 440  # Hz

In [15]:
file = 'jobs1.csv'

In [16]:
try: 
    df_more = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
    print('file successfully opened')
except:
    print('file not found, creating new file')
    df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
    df_more.to_csv(file, sep='\t', encoding='utf-8')

file successfully opened


In [17]:
df_more

,Title,Location,Company,Salary,Synopsis
0,Junior Data Scientist,"Austin, TX 78735",Kestra Financial,$84.2K - $107K a year,"Create complex data pipelines, while learning ..."
1,Data Scientist I (22-0623),"Austin, TX 78741 (Pleasant Valley area)",Office of the Attorney General of Texas,None,"Develops data quality measures, analyzes data ..."
2,Data Scientist,"Austin, TX 78701 (Downtown area)",Zilliant,$110K - $140K a year,Experience implementing data science routines ...
3,Data Scientist,"Hybrid remote in Austin, TX",YETI,$103K - $131K a year,Experience with data visualization tools (e.g....
4,Junior Data Scientist,"Austin, TX",ANALYTOS,$67.6K - $85.6K a year,Designs and drives the creation of new standar...
...,...,...,...,...,...
13043,Customer Engineer (ML/AI/Kubernetes),"Remote in Chicago, IL",Pachyderm,$113K - $143K a year,Pachyderm helps customers get their ML and AI ...
13044,Senior Machine Learning Architect,"Chicago, IL+1 location",Credera Experienced Hiring Job Board,$184K - $233K a year,Our data capabilities help our clients gain co...
13045,Staff ML Engineer - Remote,"Remote in Chicago, IL",Clari,$124K - $157K a year,A passion for answering questions by analyzing...
13046,Lead Data Engineer - Snowflake,"Remote in Rosemont, IL",Merchants Fleet,$118K - $149K a year,Experience analyzing data for data quality and...


In [18]:
headers_list = [
# Firefox 77 Mac
{
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Firefox 77 Windows
{
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Accept-Encoding": "gzip, deflate, br",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Chrome 83 Mac
{
"Connection": "keep-alive",
"DNT": "1",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "none",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
},
# Chrome 83 Windows 
{
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "same-origin",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-User": "?1",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9"
}
]

In [19]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 5000

In [ ]:
i = 0
count = 0
results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
                 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
                 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        url = url_template.format(city, start)
        headers = random.choice(headers_list)
        r = requests.Session()
        r.headers = headers
        html = r.get(url)
        #html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        #captcha detector
        if 'hCaptcha solve page' in soup.get_text():
            print('captcha detected')
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobTitle').text.replace('\n', '').replace('new', '')
            except:
                title = 'None'
            try:
                location = each.find('div', {'class':"companyLocation" }).text.replace('\n', '')
            except:
                location = 'None'
            try: 
                company = each.find(class_='companyName').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('span', {'class':'estimated-salary'}).text.replace('\n', '').replace('Estimated', '')
            except:
                salary = 'None'
            synopsis = each.find('div', {'class':'job-snippet'}).text.replace('\n', '')
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            i += 1
            if i % 1000 == 0:
                print('You have ' + str(i) + ' results. ' + str(df_more.dropna().drop_duplicates().shape[0]) + " of these aren't rubbish.")
        #count the number of pages scraped and print, so I know how far the code is running
        count+= 1
        print(count, 'loops done, proceed to sleep, current city is ', city)
        df_more.to_csv(file, sep='\t', encoding='utf-8')#write every page so i don't lose my data again
        sleep(random.randint(7,20))
        #I don't want too see long line of the process, clear_output will clear previous lines
        clear_output(wait=True)
        print(df_more.shape, 'data saved, last visited city is ', city, ', continuing....')
    print('moving to next city')
    sleep(random.randint(30,60))

(40867, 5) data saved, last visited city is  Denver , continuing....
captcha detected
